# Example program to estimate a funtion from n dimension euclid space to m dim space using 1 layer of tanh neuron system

In [1]:
import numpy as np
import tensorflow as tf
import os

data (fake x,y) :replace with actual data

In [2]:
ux=[2,4,5]
covx=[[1,0.3,0],[0.3,1,0.5],[0,0.5,2]]
n=20
A=np.transpose([[1,-1,3],[2,-1,1]])
b=np.array([3,2])

#3 dim input X
X_train=np.random.multivariate_normal(ux,covx,n)
#2d output Y
Y_train=np.matmul(X_train,A)+b
Y_train=np.array([y*100/np.sum(y) for y in Y_train])

#test data
#3 dim input X
X_test=np.random.multivariate_normal(ux,covx,n)
#2d output Y
Y_test=np.matmul(X_test,A)+b
Y_test=np.array([y*100/np.sum(y) for y in Y_train])


dimX=X_train[0].shape[0]
dimY=Y_train[0].shape[0]

In [3]:
#print("3d Xs \n",X_train)
#print("2d Ys\n",Y_train)

# neural network using tanh neurons

1st: fully connected layer neural net

In [4]:
#parameters: regularizer, number of neurons
reg_factor=0.01
batch_size=n
neuron=dimX*200
num_itr=5000
decscent_factor=0.001

In [5]:
tf.reset_default_graph()
sess=tf.InteractiveSession()

X=tf.placeholder(tf.float32,shape=[None,int(X_train.shape[-1])])
Y=tf.placeholder(tf.float32,shape=[None,int(Y_train.shape[-1])])
A=tf.Variable(tf.random_normal([int(X.shape[-1]),neuron]))
b=tf.Variable(tf.random_normal([neuron]))
L1=tf.tanh(tf.matmul(X,A)+b)
M=tf.Variable(tf.random_normal([neuron,int(Y.shape[-1])]))
l=tf.Variable(tf.random_normal([int(Y.shape[-1])]))
Y_estim=tf.matmul(L1,M)+l

#add regulerizer if data overfits and gives bad test results
Loss=tf.reduce_mean(tf.square(Y_estim-Y)) #+reg_factor*(tf.norm(M)+tf.norm(l))
optimizer=tf.train.GradientDescentOptimizer(decscent_factor)
train=optimizer.minimize(Loss)
saver = tf.train.Saver()

training:

In [6]:
sess.run(tf.global_variables_initializer())
for iter in range(num_itr):
    sess.run(train,feed_dict={X:X_train,Y:Y_train})
    if iter==num_itr-1:
        print("final_test_mean_sqr_error",sess.run(Loss,feed_dict={X:X_train,Y:Y_train}),end=",   ")
save_path = saver.save(sess, os.getcwd()+"/tmp/model.ckpt")
print("\n Model saved in path: %s" % save_path)
print("\n optimization finished \n \n model variables saved")


final_test_mean_sqr_error 0.001137124,   

check individual difference with training values, test value

In [7]:
estims=Y_estim.eval(feed_dict={X:X_train})
diff_train=Y_train-estims

estims=Y_estim.eval(feed_dict={X:X_test})
diff_test=Y_test-estims

In [8]:
diff_train #difference from training values

array([[ 3.48491677e-04,  4.82368383e-05],
       [-9.79037445e-04,  4.44366145e-05],
       [ 3.43643728e-03, -1.84407859e-04],
       [ 4.08658747e-03, -4.94489435e-03],
       [-6.57067134e-02,  6.19377925e-02],
       [ 4.16305544e-02,  6.12747191e-02],
       [ 4.58882594e-02,  2.40885473e-02],
       [-4.01984158e-02, -7.90986118e-02],
       [-8.58879162e-03,  4.09507824e-03],
       [-4.90739835e-03, -2.10420798e-02],
       [ 3.44817095e-03, -2.03864031e-03],
       [-4.83983960e-03,  4.79406323e-03],
       [ 4.36813509e-03,  1.35704788e-02],
       [ 1.06742890e-03,  1.33692929e-02],
       [ 1.11302724e-01, -2.43142749e-02],
       [-1.10120849e-03, -8.82463379e-03],
       [-7.01781611e-04,  7.36113887e-04],
       [ 1.73157823e-04,  3.93527113e-03],
       [-6.11610736e-02, -5.13457928e-02],
       [-2.73264688e-02,  5.27751856e-03]])

In [9]:
diff_test #difference from test value

array([[-12.63749559,  16.08571489],
       [  6.42188015,  -4.44330731],
       [ 31.06132447, -17.3656572 ],
       [ -5.67065706,   4.36962214],
       [ -1.33692406,   1.03590439],
       [ -1.5503952 ,  -0.45228653],
       [  6.8724416 ,  -6.48222478],
       [  4.35413066,  -7.33583537],
       [ -9.30611229,   9.64658547],
       [ -3.16202715,   3.13414171],
       [ 16.70449798, -17.19916884],
       [  7.91374689,   9.35946173],
       [  4.66003067,  -4.08932907],
       [ -5.18753883,  -2.77671745],
       [  4.43306481,  -5.50963341],
       [ 16.93805056, -17.89010363],
       [ -6.7842406 ,   6.161188  ],
       [ -0.24563068,  -0.54021222],
       [  8.84305188,  -9.59953686],
       [ 13.97853672, -12.36347782]])